In [1]:
import os
os.chdir("../tsp")
print("Current Working Directory:", os.getcwd())

import time
import torch
from torch.distributions import Categorical, kl
# from d2l.torch import Animator

from net import Net
from aco import ACO
from utils import gen_pyg_data, load_test_dataset
from greedy import test_greedy, GreedyTSP
from tqdm import tqdm

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Current Working Directory: /Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/tsp
Using device: cpu


In [2]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse=None):
    if model:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO(
        n_ants=n_ants,
        heuristic=heu_mat,
        distances=distances,
        device=device
        )
    
    else:
        aco = ACO(
        n_ants=n_ants,
        distances=distances,
        device=device
        )
        if k_sparse:
            aco.sparsify(k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results

@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for pyg_data, distances in tqdm(dataset, desc="Testing Deep ACO Algorithm"):
    # for pyg_data, distances in dataset:
        results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start


In [ ]:
torch.set_printoptions(precision=3, sci_mode=False)
# Configuration
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)

# # Testing Deep ACO Model
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('Deep ACO Model:')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    print(f"T={t}, average cost is {avg_aco_best[i]}.")

/Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/tsp/utils.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_tensor = torch.load(f'../data/tsp/testDataset

In [ ]:
# Testing Greedy Algorithm
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=True)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")